<a href="https://colab.research.google.com/github/mmaruthi/Assignment5/blob/master/CNN-V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


#from keras.applications import VGG16
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import os
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50


In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()   # To Return the top 5 rows of the data frame 

In [0]:
df.describe()  # To get the summary of statistics

In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True,augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
#       'items' is a df generated from main df based on batch_size. 
#       items.iterrows() will take item['image_path'] and fetch it to cv2.imread().
#       cv2.imread() will convert it to a numpy array which can be used by python
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        # target is a dictionary with all gender cols counted against key 'gender_output' and so on.
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
## Cut out 
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [0]:
# Creating Train and Validation data frames 
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15)
train_df.shape, val_df.shape

In [0]:
# Display top two rows from train and val data frames 
train_df.head(2) 
val_df.head(2)  

In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32, 
                                augmentation=ImageDataGenerator(rescale=1./255,
                                                                featurewise_center=True,
                                                                featurewise_std_normalization=True,
                                                                horizontal_flip=True,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.2,
                                                                fill_mode='nearest',
                                                                rotation_range=90,
                                                                preprocessing_function=get_random_eraser(v_l=1, v_h=1)))

valid_gen = PersonDataGenerator(val_df, batch_size=32, shuffle=False,
                                augmentation=ImageDataGenerator(rescale=1./255,
                                                                featurewise_center=True,
                                                                featurewise_std_normalization=True))

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
# targets is a dictionary that we get from person data generator.
# k will get key value 'age_output' which will be split to get 'age' only
# v will get value of disctionary. We will take how many columns are there in v to understand classes within each class. eg: v.shape[1] will be 2
# for key 'gender_output' because there are 2 sub-categories 'male' & 'female'
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

In [0]:
# To check if images are getting read correctly
from google.colab.patches import cv2_imshow
img = cv2.imread('resized/1.jpg',0)
cv2_imshow(img)
print('type::',type(img))

In [0]:
#Using Sequential model 

from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import os

backbone = Sequential()

backbone.add(Convolution2D(16, (3, 3),use_bias=False,input_shape=(224, 224, 3))) #222
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(Convolution2D(16, (3, 3),use_bias=False)) #220
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(Convolution2D(32, (3, 3),use_bias=False)) #218
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(Convolution2D(32, (3, 3),use_bias=False)) #216
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(Convolution2D(64, (3, 3),use_bias=False)) #214
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(Convolution2D(64, (3, 3),use_bias=False)) #212
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(MaxPooling2D(pool_size=(2, 2)))  #106

backbone.add(Convolution2D(64, (3, 3),use_bias=False)) #104
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(Convolution2D(64, (3, 3),use_bias=False)) #102
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(Convolution2D(96, (3, 3),use_bias=False)) #100
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(Convolution2D(96, (3, 3),use_bias=False)) #98
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(Convolution2D(96, (3, 3),use_bias=False)) #96
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(MaxPooling2D(pool_size=(2, 2)))  #48

backbone.add(Convolution2D(128, (3, 3),use_bias=False)) #46
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(Convolution2D(128, (3, 3),use_bias=False)) #44
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(Convolution2D(128, (3, 3),use_bias=False)) #42
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(Convolution2D(128, (3, 3),use_bias=False)) #40
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(Convolution2D(128, (3, 3),use_bias=False)) #38
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(MaxPooling2D(pool_size=(2, 2)))  #19

backbone.add(Convolution2D(128, (3, 3),strides=(2,2))) #9
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))

backbone.add(Convolution2D(128, (3, 3),dilation_rate=(2, 2))) #5
backbone.add(BatchNormalization())
backbone.add(Activation('relu'))
backbone.add(Dropout(0.2))
backbone.add(GlobalAveragePooling2D(name='avg_pool'))

neck = backbone.output

def build_tower(in_layer):
    return neck

def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)
model.summary()

In [0]:
from keras.utils import plot_model
#plot_model(model, show_shapes=True,to_file='model.png')
plot_model(model)

In [0]:
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
def scheduler(epoch, lr):
  return round(0.004 * 1/(1 + 0.319 * epoch), 10)
opt = SGD(lr=0.003, momentum=0.9)
#opt = SGD(momentum=0.5)
model.compile(
    optimizer=opt,
    loss = "binary_crossentropy",
    metrics=["accuracy"]
)

In [0]:
# model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=10)
model.summary()

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
 #  use_multiprocessing=False,
 #  workers=1, 
    epochs=100,
    verbose=1,
    callbacks=[LearningRateScheduler(scheduler,verbose=1)])

In [0]:
model